In [14]:
from datastore import DataStore as ds


ds.config.enable_debug()


nat = ds.from_file('../tests/dataset/users.csv')
nat.sql("select 1").execute()

nat = nat.filter(nat.age < 35)

nat['age_minus_10'] = nat['age'] - 10

nat = nat.add_prefix('col_').filter(nat.col_age > 25).filter(nat.col_country == 'USA')

nat['doubled'] = nat['col_age'] * 2

nat2 = nat.sql("doubled > 35")
nat2 = nat2.filter(nat2.doubled > 25)
nat2["float_age"] = nat2.doubled.cast("Float64")
print('Executing...')
nat2.explain()

nat2["col_age"]
# df = nat2.to_df()
# print(df)

D Cache invalidated: version now 1
D ======================================================================
D DataStore.execute() called
D ======================================================================
D Not connected, establishing connection...
D Connecting to data source (database=:memory:)...
D Created connection for table function
D [chDB] Connected to database: :memory:
D Connection established successfully
D Testing data source accessibility: SELECT 1 FROM file('../tests/dataset/users.csv', 'CSVWithNames') LIMIT 1
D ======================================================================
D [chDB] Connection execution
D ----------------------------------------------------------------------
D [chDB] Output format: Dataframe
D [chDB] SQL:
D   SELECT 1 FROM file('../tests/dataset/users.csv', 'CSVWithNames') LIMIT 1
D ======================================================================
D [chDB] Result: 1 rows x 1 cols
D [chDB] Query time: 1.30ms
D Data source is accessible
D G

Executing...
Execution Plan (in execution order)

 [1] 📊 Data Source: file('../tests/dataset/users.csv', 'CSVWithNames')

Operations:
────────────────────────────────────────────────────────────────────────────────
    ️  Segment 1 [chDB] (from source): Operations 2-3
    ️  Segment 2 [Pandas] (on DataFrame): Operations 4-4
    ️  Segment 3 [chDB] (on DataFrame): Operations 5-7
    ️  Segment 4 [Pandas] (on DataFrame): Operations 8-8
    ️  Segment 5 [chDB] (on DataFrame): Operations 9-10
    ️  Note: SQL operations after Pandas ops use Python() table function

 [2] 🚀 [chDB] WHERE: "age" < 35
 [3] 🐼 [Pandas] Assign column 'age_minus_10' = ("age"-10)
 [4] 🐼 [Pandas] Add prefix 'col_' to all columns
 [5] 🚀 [chDB] WHERE: "col_age" > 25
 [6] 🚀 [chDB] WHERE: "col_country" = 'USA'
 [7] 🐼 [Pandas] Assign column 'doubled' = ("col_age"*2)
 [8] 🚀 [chDB] SQL: doubled > 35 (expanded)
 [9] 🚀 [chDB] WHERE: "doubled" > 25
 [10] 🚀 [chDB] Assign column 'float_age' = CAST("doubled",'Float64')

─────────

0    28
2    31
4    29
Name: col_age, dtype: int64

In [15]:
nat2["col_name"].str.upper()
nat2.to_df()["col_name"].str.upper()
# nat2["col_name"].upper()


D to_df() called - triggering execution
D Using cached result (version=0)


0     ALICE SMITH
2    DIANA PRINCE
4       GRACE LEE
Name: col_name, dtype: object

In [16]:
# Aggregate functions
from datastore.expressions import col
agg_ds = ds.from_file('../tests/dataset/orders.csv')
print(agg_ds.head())

agg_ds.groupby('product_id').agg(
    new_col=col('product_id') * 2, # meaningless column, just for testing
    total_quantity=col('quantity').sum(),
    total_revenue=agg_ds.amount.sum(),
    stddev=col('amount').stddev_samp(),
    stddev2=agg_ds.amount.stddev_samp(),
).to_df()

D Cache invalidated: version now 1
D ======================================================================
D Starting execution (version=1)
D ======================================================================
D Lazy operations chain (1 operations):
D   [1] LIMIT: 5
D Connecting to data source (database=:memory:)...
D Created connection for table function
D [chDB] Connected to database: :memory:
D Connection established successfully
D Discovering schema for table function: DESCRIBE file('../tests/dataset/orders.csv', 'CSVWithNames') SETTINGS session_timezone='UTC'
D ======================================================================
D [chDB] Connection execution
D ----------------------------------------------------------------------
D [chDB] Output format: Dataframe
D [chDB] SQL:
D   DESCRIBE file('../tests/dataset/orders.csv', 'CSVWithNames') SETTINGS session_timezone='UTC'
D ======================================================================
D [chDB] Result: 6 rows x 7 col

   order_id  user_id  product_id  quantity order_date  amount
0      1001        1         101         2 2024-01-10   59.98
1      1002        2         102         1 2024-01-15  129.99
2      1003        1         103         3 2024-01-20   89.97
3      1004        3         101         1 2024-01-25   29.99
4      1005        4         104         2 2024-02-01  199.98


,product_id,new_col,total_quantity,total_revenue,stddev,stddev2
0,104,208,5,499.95,57.729253,57.729253
1,105,210,4,319.96,113.122943,113.122943
2,101,202,8,239.92,42.412265,42.412265
3,103,206,6,179.94,29.990000,29.990000
4,102,204,4,519.96,75.049761,75.049761


In [ ]:
ds.config.disable_debug()
ch_ds = ds.from_clickhouse("sql-clickhouse.clickhouse.com", "pypi", "pypi_downloads", "demo", "", secure=True)

print(ch_ds.head(10))

                 project  count
0                      0      1
1  2captcha-python-async      1
2                   2to3      1
3                  3dfin      1
4     a-pandas-ex-xml2df      1
5                a2a-sdk      7
6                   a2ml      1
7                   a2p2      1
8                 a2wsgi     36
9                    a7p      1
